# Asistente de Codificación Cualitativa con GPT-4o y Exportación CSV (versión corregida)
Este cuaderno permite cargar entrevistas, obtener sugerencias de codificación usando GPT-4o y exportarlas a un archivo `.csv`. Ahora incluye limpieza automática del JSON generado.

## 1. Instalación de dependencias

In [1]:
!pip install openai pandas

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


## 2. Cargar archivos de entrevistas

In [3]:
import os
import pandas as pd

carpeta = "/content/drive/MyDrive/_Luz Stella Doctorado/Mario Chamie Mazzilli/1_Audios_Archivos .m4a/Prueba_Piloto_3_con_IA_Assist"
archivos = [f for f in os.listdir(carpeta) if f.endswith(".txt")]
print("Archivos disponibles:", archivos)

def leer_entrevista(nombre_archivo):
    with open(os.path.join(carpeta, nombre_archivo), encoding="utf-8") as f:
        return f.read()

contenido = leer_entrevista(archivos[0])
print(contenido[:1000])


Archivos disponibles: ['comunidad_voz_036.txt', 'institucion_voz_035.txt', 'comunidad_prueba_01.txt']
 Formulario de entrevistas se mistrucuturada comunidades. Tesis y doctoral, ciudadanía participación y capacidades en la formulación de los PEDED en la su región de Montedemaria. Autor, Mario Chamií Masilo, Universidad Esternáe Colombia. Versión adoptada, adaptada para actores comunitarios. El objetivo en el alde de instrumento es recoger experiencias, valoraciones y propuestas de actores clave sobre la participación comunitaria y el desarrollo de capacidades en la fase de la formulación de los PEDED y PEDED en Montedemaria. Primero, muchas gracias por la participación de S.A.1. En antestrevista, el objetivo es conocer su percepción sobre la participación de su comunidad en la formulación de los PEDED. Su opinión es muy valiosa para una investigación académica que busca mejorar. Los procesos de planeación participativo en el país, esta conversación será confiencial y usted puede retira

## 3. Ingreso de clave de API y configuración del cliente

## 🔐 Guía para obtener la clave API de OpenAI

Para que este notebook funcione correctamente, debes generar tu propia **clave secreta (API key)** desde OpenAI. A continuación, los pasos para obtenerla:

---

### 1. Crear una cuenta en OpenAI
- Visita: [https://platform.openai.com/signup](https://platform.openai.com/signup)
- Regístrate con tu correo electrónico o cuenta de Google/Microsoft.

---

### 2. Iniciar sesión en la plataforma
- Ve a: [https://platform.openai.com/account/api-keys](https://platform.openai.com/account/api-keys)
- Inicia sesión con tu cuenta ya creada.

---

### 3. Generar una nueva clave
- Haz clic en **“+ Create new secret key”**
- Asigna un nombre identificador (por ejemplo: `notebook_tesis`)
- Haz clic en **“Create secret key”**
- **Copia la clave** que se genera (ejemplo: `sk-XXXX...`)
  - ⚠️ Solo se muestra una vez. Guárdala en lugar seguro.

---

### 4. Activar un plan de uso (si aún no tienes uno)
- Visita: [https://platform.openai.com/account/billing/overview](https://platform.openai.com/account/billing/overview)
- Selecciona el plan **“Pay as you go”**
- Agrega un método de pago válido
- Puedes establecer límites mensuales y recargas automáticas

---

### 5. Ingresar la clave en el notebook
Cuando el sistema la solicite, pégala donde aparece:

```python
clave = getpass.getpass("Ingrese su clave de OpenAI: ")
```

No se mostrará en pantalla y solo se usará para tu sesión local.

---

### 🔒 Recomendaciones de seguridad
- Nunca compartas tu clave públicamente
- No la subas a GitHub ni a repositorios públicos
- Si crees que fue comprometida, elimínala y genera una nueva desde tu cuenta

---


In [ ]:
from openai import OpenAI
import getpass

# clave = getpass.getpass("Ingrese su clave de OpenAI: ")
clave = "PON_AQUI_TU_CLAVE_DE_OPENAI"
client = OpenAI(api_key=clave)
# (clave eliminada por seguridad)

Ingrese su clave de OpenAI: ··········


## 4. Función para sugerir codificación con JSON limpio

In [5]:
def sugerir_codigos_chatgpt(texto, sistema="Eres un investigador cualitativo experto en codificación temática."):
    prompt_usuario = f"""A partir del siguiente fragmento de entrevista, propón códigos cualitativos y su descripción.

Texto:
{texto}

Entrega ÚNICAMENTE la lista JSON. No escribas explicaciones ni encabezados. La respuesta debe ser directamente una lista de objetos JSON con campos: 'codigo', 'descripcion'."""

    respuesta = client.chat.completions.create(
        model="gpt-4o",
        messages=[
            {"role": "system", "content": sistema},
            {"role": "user", "content": prompt_usuario}
        ],
        temperature=0.7
    )

    return respuesta.choices[0].message.content


## 5. Probar codificación en un fragmento

In [6]:
fragmento = contenido[:1500]
respuesta_json = sugerir_codigos_chatgpt(fragmento)
print(respuesta_json)


```json
[
    {
        "codigo": "participacion_comunitaria",
        "descripcion": "Involucramiento de la comunidad en la formulación de los Programas de Desarrollo con Enfoque Territorial (PEDED) en Montedemaria."
    },
    {
        "codigo": "desarrollo_capacidades",
        "descripcion": "Desarrollo de habilidades y capacidades de los actores comunitarios en el contexto de la planificación participativa de los PEDED."
    },
    {
        "codigo": "percepcion_comunitaria",
        "descripcion": "Opiniones y valoraciones de los miembros de la comunidad sobre su participación en los procesos de planificación del desarrollo."
    },
    {
        "codigo": "consentimiento_informado",
        "descripcion": "Proceso de obtención de permiso de los participantes para ser entrevistados y grabados, asegurando la confidencialidad y la opción de retirarse en cualquier momento."
    },
    {
        "codigo": "caracterizacion_participante",
        "descripcion": "Datos demográficos y 

## 6. Limpieza de respuesta y exportación a CSV

In [7]:
import json
import re

try:
    # Extraer solo la parte JSON entre corchetes
    json_puro = re.search(r"\[.*\]", respuesta_json, re.DOTALL).group(0)
    datos = json.loads(json_puro)

    if isinstance(datos, list):
        df_codigos = pd.DataFrame(datos)
        nombre_archivo = "codigos_" + archivos[0].replace(".txt", ".csv")
        df_codigos.to_csv(nombre_archivo, index=False)
        print(f"✅ Archivo exportado: {nombre_archivo}")
        display(df_codigos)
    else:
        print("⚠️ La respuesta no es una lista válida de códigos.")

except Exception as e:
    print("❌ Error al procesar la respuesta como JSON:")
    print(e)


✅ Archivo exportado: codigos_comunidad_voz_036.csv


,codigo,descripcion
0,participacion_comunitaria,Involucramiento de la comunidad en la formulac...
1,desarrollo_capacidades,Desarrollo de habilidades y capacidades de los...
2,percepcion_comunitaria,Opiniones y valoraciones de los miembros de la...
3,consentimiento_informado,Proceso de obtención de permiso de los partici...
4,caracterizacion_participante,Datos demográficos y de identificación de los ...
5,enfoque_diferencial,Consideración de las características individua...
6,rol_comunitario,Posición o función que ocupa el participante d...
